In [ ]:
#!/usr/bin/env python3
# coding=utf-8
"""
DOFBOT Arm Control – Fruit Sorting Test with AI
"""


In [ ]:
import time
from Arm_Lib import Arm_Device

print('Initialising DOFBOT arm...')
Arm = Arm_Device()
time.sleep(0.3)
print('DOFBOT initialised.')


In [ ]:
def arm_clamp_block(enable: int) -> None:
    """Open (0) or close (1) the DOFBOT gripper."""
    if enable == 0:
        Arm.Arm_serial_servo_write(6, 60, 400)
    else:
        Arm.Arm_serial_servo_write(6, 135, 400)
    time.sleep(0.4)

def arm_move(poses, t: int = 700) -> None:
    """Move servos 1–5 to the angles defined in poses."""
    for i in range(5):
        servo_id = i + 1
        angle = poses[i]
        move_time = int(t * 1.2) if servo_id == 5 else t
        Arm.Arm_serial_servo_write(servo_id, angle, move_time)
        time.sleep(0.01)
    time.sleep(t / 1000.0)

def arm_move_up() -> None:
    """Lift the arm to a neutral 'up' position."""
    Arm.Arm_serial_servo_write(2, 90, 1500)
    Arm.Arm_serial_servo_write(3, 90, 1500)
    Arm.Arm_serial_servo_write(4, 90, 1500)
    time.sleep(0.3)

print('Helper functions loaded.')


In [ ]:
p_home = [90, 130, 0, 0, 90]
p_pick_top = [90, 80, 50, 50, 270]
p_pick = [90, 53, 33, 36, 270]

p_basket_yellow = [65, 22, 64, 56, 270]
p_basket_red    = [117, 19, 66, 56, 270]
p_basket_green  = [136, 66, 20, 29, 270]
p_basket_blue   = [44, 66, 20, 28, 270]

FRUIT_TO_BASKET = {
    'apple':        p_basket_green,
    'banana':       p_basket_yellow,
    'orange':       p_basket_red,
    'lemon':        p_basket_blue,
    'pomegranate':  p_basket_red,
    'tomato':       p_basket_green,
}

print('Poses and fruit mapping configured.')


In [ ]:
def grab_from_pick_and_place(target_pose, label: str = '') -> None:
    """Execute a full pick-and-place cycle for one fruit."""
    if target_pose is None:
        print('No valid basket pose. Aborting.')
        return

    print(f'Sorting fruit: {label}')

    arm_clamp_block(0)
    arm_move(p_home, 1000)

    arm_move(p_pick_top, 1000)
    arm_move(p_pick, 1000)

    arm_clamp_block(1)
    arm_move(p_pick_top, 1000)

    arm_move(target_pose, 1100)

    arm_clamp_block(0)

    arm_move_up()
    arm_move(p_home, 1100)

    print('Sorting cycle completed.')


In [ ]:
print('Manual test: type a fruit name (apple/banana/orange/lemon/pomegranate/tomato) or q to quit.')

try:
    while True:
        name = input('Fruit to sort: ').strip().lower()
        if name == 'q':
            break
        if not name:
            continue

        pose = FRUIT_TO_BASKET.get(name)
        if pose is None:
            print('No basket mapping for this fruit.')
            continue

        grab_from_pick_and_place(pose, label=name)

except KeyboardInterrupt:
    print('Interrupted by user.')

finally:
    try:
        del Arm
    except Exception:
        pass
    print('DOFBOT released.')
